<a href="https://colab.research.google.com/github/sunandhini96/TSAI_ERAV1/blob/main/Capstone/Stage_1/stage_1_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/capstone

/content/drive/MyDrive/capstone


In [3]:
!git clone https://github.com/sunandhini96/TSAI_ERAV1.git

fatal: destination path 'TSAI_ERAV1' already exists and is not an empty directory.


In [3]:
cd /content/drive/MyDrive/capstone/TSAI_ERAV1/Capstone/Stage_1

/content/drive/MyDrive/capstone/TSAI_ERAV1/Capstone/Stage_1


In [4]:
!pip install -r requirements.txt

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-1pwyohqx
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-1pwyohqx
  Resolved https://github.com/m-bain/whisperx.git to commit 8227807fa9e076901ea4b4fbbf79c9777a6f5e03
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/SYSTRAN/faster-whisper.git (to revision 0.10.0) to /tmp/pip-install-pna9d_73/faster-whisper_652aa4424ab940e4a0794fee10c50626
  Running command git clone --filter=blob:none --quiet https://github.com/SYSTRAN/faster-whisper.git /tmp/pip-install-pna9d_73/faster-whisper_652aa4424ab940e4a0794fee10c50626
  Running command git checkout -q e1a218fab1ab02d637b79565995bf1a9c4c83a09
  Resolved https://github.com/SYSTRAN/faster-whisper.git to commit e1a218fab1ab02d637b79565995bf1a9c4c83a09
  Preparing metadata (setup.py) ... done


In [5]:
!pip install glob2

In [7]:
import glob2
import torch
from network import CLIPPhi2Model, train_model, frange_cycle_linear
from dataset import collate_fn, llavadataset
from torch.utils.data import random_split, DataLoader
import wandb
import torch.nn as nn
from transformers import AutoTokenizer
import pickle
#import bitsandbytes as bnb
import gc


def main():
    with open("/content/drive/MyDrive/capstone/coco_captions.pickle", "rb") as fp:   # Unpickling
        coco_unpickle = pickle.load(fp)

    clip_model_name = "wkcn/TinyCLIP-ViT-61M-32-Text-29M-LAION400M"
    phi_model_name  = "microsoft/phi-2"
    train_batch_size = 2
    val_batch_size   = 4
    device     = 'cuda'
    tokenizer  = AutoTokenizer.from_pretrained(phi_model_name, trust_remote_code=True)

    # model
    MModalGPT        = CLIPPhi2Model().to(device)
    max_steps        = 20000
    model_save_step  = 1000
    model_val_step   = 1000
    log_step         = 1000
    max_token_filter = 35

    # data loaders
    train_dataloader = DataLoader(llavadataset(coco_unpickle, phi_model_name,clip_model_name,'train',tokenizer),
                      collate_fn=collate_fn, batch_size=train_batch_size, num_workers = 10, shuffle=True, pin_memory=True)

    val_dataloader   = DataLoader(llavadataset(coco_unpickle, phi_model_name,clip_model_name,'val',tokenizer),
                      collate_fn=collate_fn, batch_size=val_batch_size, num_workers = 10, shuffle=True, pin_memory=True)

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, MModalGPT.parameters()), lr=1e-4)
    train_model(MModalGPT, train_dataloader, val_dataloader, optimizer, device, max_steps,model_save_step,model_val_step,log_step,max_token_filter,tokenizer)



In [8]:
if __name__ == "__main__":
    wandb.init(project="capstone_multimodal_gpt_project", name="step1_projector_pretrain") #,log_model='all')
    torch.cuda.amp.autocast(enabled=True)
    torch.cuda.empty_cache()
    gc.collect()
    torch.set_float32_matmul_precision('medium')
    main()

wandb: Currently logged in as: gsunandhini. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Train size 532577 and validation size 59176
Train size 532577 and validation size 59176
Training started.
Step 0/20000: Avg Running Loss = 9.10235595703125
Saving Checkpoint for step :  1000
0 - Target captions:
 A tan stuffed animal wearing a green bandana.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>  
0 - predicted_captions:
 A woman is a man in a suit and tie sitting on a couch in a living room with<|endoftext|> 
1 - Target captions:
 A woman sitting at a table with a pair of scissors cutting paper.<|endoftext|>  
1 - predicted_captions:
 A woman is a man in a suit and a man in a suit and a woman in a<|endoftext|> 
2 - Target captions:
 Young woman demonstrating a refrigerator in an appliance store.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>  
2 - predicted_captions:
 A woman is a part of a group of people standing in front of a building. The building<|endoftext|> 
3 - Target captions:
 A male tennis player on a court with a tennis racket i